In [49]:
#Copyright

#@InProceedings{wang2017chestxray,
#author = {Wang, Xiaosong and Peng, Yifan and Lu, Le and Lu, Zhiyong and Bagheri, Mohammadhadi
#and Summers, Ronald},
#title = {ChestX-ray8: Hospital-scale Chest X-ray Database and Benchmarks on Weakly-Supervised
#Classification and Localization of Common Thorax Diseases},
#booktitle = {2017 IEEE Conference on Computer Vision and Pattern Recognition (CVPR)},
#pages = {3462--3471},
#year = {2017}
#}

#We acknowledge the NIH Clinical Center for providing the dataset used in this study.
#https://nihcc.app.box.com/v/ChestXray-NIHCC/folder/36938765345

#Paper

#Wang, X., Peng, Y., Lu, L., Lu, Z., Bagheri, M., & Summers, R.
#M. (2017). Chestx-ray8: Hospital-scale chest x-ray database
#and benchmarks on weakly-supervised classification and
#localization of common thorax diseases. In Proceedings of the
#IEEE conference on computer vision and pattern recognition
#(pp. 2097-2106).

In [12]:
import os
import shutil
import pandas as pd
from sklearn.model_selection import train_test_split
data = pd.read_csv('D:\Academia\Matematicas Aplicadas\Semestre7\Proyecto1\ProyectoCNN\CHST-X8\Data_Entry_2017_v2020.csv')
data['Finding Labels'].value_counts()[:40]

Finding Labels
No Finding                            60361
Infiltration                           9547
Atelectasis                            4215
Effusion                               3955
Nodule                                 2705
Pneumothorax                           2194
Mass                                   2139
Effusion|Infiltration                  1603
Atelectasis|Infiltration               1350
Consolidation                          1310
Atelectasis|Effusion                   1165
Pleural_Thickening                     1126
Cardiomegaly                           1093
Emphysema                               892
Infiltration|Nodule                     829
Atelectasis|Effusion|Infiltration       737
Fibrosis                                727
Edema                                   628
Cardiomegaly|Effusion                   484
Consolidation|Infiltration              441
Infiltration|Mass                       420
Effusion|Pneumothorax                   403
Effusion|Mass    

In [31]:
data = pd.read_csv('D:\Academia\Matematicas Aplicadas\Semestre7\Proyecto1\ProyectoCNN\CHST-X8\Data_Entry_2017_v2020.csv')
source_folder = r'D:\Academia\Matematicas Aplicadas\Semestre7\Proyecto1\ProyectoCNN\CHST-X8\ImagenesRadiografias'

#Enfermedades con las que trabajaremos
diseases = {"No Finding", "Infiltration", "Effusion", "Atelectasis", "Nodule", "Mass", "Pneumothorax", "Consolidation", "Pleural_Thickening", "Cardiomegaly"}

#Obtener las filas con una sola enfermedad y dentro de las categorías permitidas
#Eliminamos las filas cuyo nombre de foto no se encuentre entre las descargadas
filtered_data = data[data['Finding Labels'].isin(diseases)]
image_files = set(os.listdir(source_folder)) 
filtered_data = filtered_data[filtered_data['Image Index'].isin(image_files)]

#Limitar cada categoría a un máximo de 1000 imágenes
df = pd.DataFrame()
for disease in diseases:
    disease_data = filtered_data[filtered_data['Finding Labels'] == disease]
    limited_disease_data = disease_data.sample(n=min(1000, len(disease_data)), random_state=42)
    df = pd.concat([df, limited_disease_data])

# Guardar el dataset filtrado en un archivo CSV
df.to_csv('disease_balanced.csv', index=False)

print(f"Total de imágenes después del filtrado: {df.shape[0]}")
print("\nDistribución de clases:")
print(df['Finding Labels'].value_counts())

Total de imágenes después del filtrado: 10000

Distribución de clases:
Finding Labels
Pleural_Thickening    1000
Cardiomegaly          1000
Effusion              1000
Atelectasis           1000
Mass                  1000
Consolidation         1000
Pneumothorax          1000
Infiltration          1000
Nodule                1000
No Finding            1000
Name: count, dtype: int64


In [74]:
train_data = []
validation_data = []
test_data = []

# Dividir por categoría
for disease in diseases:
    disease_data = df[df['Finding Labels'] == disease]
    #Dividir en train, validation y test
    train, temp = train_test_split(disease_data, train_size=0.7, random_state=42)
    validation, test = train_test_split(temp, train_size=0.66, random_state=42)
    # Agregar a los conjuntos principales
    train_data.append(train)
    validation_data.append(validation)
    test_data.append(test)

train_data = pd.concat(train_data)
validation_data = pd.concat(validation_data)
test_data = pd.concat(test_data)
print(f"Tamaño de entrenamiento: {train_data.shape[0]}")
print(f"Tamaño de validación: {validation_data.shape[0]}")
print(f"Tamaño de prueba: {test_data.shape[0]}")

# Guardar los conjuntos divididos en archivos CSV
train_data.to_csv('train_data.csv', index=False)
validation_data.to_csv('validation_data.csv', index=False)
test_data.to_csv('test_data.csv', index=False)

Tamaño de entrenamiento: 7000
Tamaño de validación: 1980
Tamaño de prueba: 1020


In [102]:
#Rutas de los CSV
train_csv = r'D:\Academia\Matematicas Aplicadas\Semestre7\Proyecto1\ProyectoCNN\CHST-X8\Filtrado\train_data.csv'
validation_csv = r'D:\Academia\Matematicas Aplicadas\Semestre7\Proyecto1\ProyectoCNN\CHST-X8\Filtrado\validation_data.csv'
test_csv = r'D:\Academia\Matematicas Aplicadas\Semestre7\Proyecto1\ProyectoCNN\CHST-X8\Filtrado\test_data.csv'

#Ruta con todas las imágenes
source_folder = r'D:\Academia\Matematicas Aplicadas\Semestre7\Proyecto1\ProyectoCNN\CHST-X8\ImagenesRadiografias' 

# Crear carpetas para cada conjunto
train_folder = r'D:\Academia\Matematicas Aplicadas\Semestre7\Proyecto1\ProyectoCNN\CHST-X8\Filtrado\train'
validation_folder = r'D:\Academia\Matematicas Aplicadas\Semestre7\Proyecto1\ProyectoCNN\CHST-X8\Filtrado\validation'
test_folder = r'D:\Academia\Matematicas Aplicadas\Semestre7\Proyecto1\ProyectoCNN\CHST-X8\Filtrado\test'

#Mandar las imágenes a cada carpeta
def copy_images(csv_file, dest_folder):
    df = pd.read_csv(csv_file)
    for image_name in df['Image Index']:
        source_path = os.path.join(source_folder, image_name)
        dest_path = os.path.join(dest_folder, image_name)
        # Copiar solo si existe el archivo
        if os.path.exists(source_path):
            shutil.copy(source_path, dest_path)
        else:
            print(f"Imagen no encontrada: {image_name}")

#Copiar las imágenes para cada conjunto
copy_images(train_csv, train_folder)
copy_images(validation_csv, validation_folder)
copy_images(test_csv, test_folder)

print("Imágenes copiadas exitosamente en las carpetas correspondientes.")

Imágenes copiadas exitosamente en las carpetas correspondientes.


In [99]:
#Rutas de los CSV
train_csv = pd.read_csv(r'D:\Academia\Matematicas Aplicadas\Semestre7\Proyecto1\ProyectoCNN\CHST-X8\Filtrado\train_data.csv')
validation_csv = pd.read_csv(r'D:\Academia\Matematicas Aplicadas\Semestre7\Proyecto1\ProyectoCNN\CHST-X8\Filtrado\validation_data.csv')
test_csv = pd.read_csv(r'D:\Academia\Matematicas Aplicadas\Semestre7\Proyecto1\ProyectoCNN\CHST-X8\Filtrado\test_data.csv')

In [100]:
print(train_csv.shape)
print(validation_csv.shape)
print(test_csv.shape)

(7000, 11)
(1980, 11)
(1020, 11)


In [101]:
test_csv.head(10000)

,Image Index,Finding Labels,Follow-up #,Patient ID,Patient Age,Patient Gender,View Position,OriginalImage[Width,Height],OriginalImagePixelSpacing[x,y]
0,00002861_000.png,Pleural_Thickening,0,2861,51,M,PA,2500,2048,0.171,0.171
1,00021035_035.png,Pleural_Thickening,12,21035,34,M,PA,2500,2048,0.168,0.168
2,00011548_021.png,Pleural_Thickening,21,11548,45,M,PA,2930,2991,0.143,0.143
3,00002758_000.png,Pleural_Thickening,0,2758,56,F,PA,2048,2500,0.171,0.171
4,00017814_003.png,Pleural_Thickening,2,17814,64,M,PA,2892,2948,0.143,0.143
...,...,...,...,...,...,...,...,...,...,...,...
1015,00014474_000.png,No Finding,1,14474,6,M,PA,2544,3056,0.139,0.139
1016,00019421_002.png,No Finding,2,19421,7,F,PA,2446,2101,0.143,0.143
1017,00015847_000.png,No Finding,3,15847,46,F,AP,3056,2544,0.139,0.139
1018,00008046_013.png,No Finding,0,8046,31,F,PA,2048,2500,0.168,0.168
